# Project: Web Traffic Forecasting
Ruoxin Jiang and Bingyan Hu
## Overview
___
### Task
The goal of our project is to forecast web traffic time series for online pages. Forecasting time series is challenging since we need to combine its seasonality, trend and other factors intelligently in modeling; the historical data itself is insufficient to capture uncertainty in future events. 

We present a hierchical time series forecasting model using Edward and demostrate three rounds of Box's loop below.

+ Hypothesis

### Data Source
We obtain real time series data from [a recent Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting). Each time series represents daily page views of a particular Wikipedia article from **07/01/2015** to **09/10/2017**. 

The model is trained on data before **07/10/2017** and we forecast number of page visits in last 60 days from **07/10/2017** to **09/10/2017**.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import edward as ed
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta

from models import *
from utils import *
from pipeline import *
from cross_validation import cross_validation

%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (24, 12)
matplotlib.rcParams['lines.linewidth'] = 2
matplotlib.rcParams['xtick.labelsize'] = 18
matplotlib.rcParams['ytick.labelsize'] = 18
matplotlib.rcParams['xtick.color'] = 'w'

ed.set_seed(42)

## Round 1: 
___
### 1.Data
We randomly pick a wikipedia article data with `ds` and `views` in long format.

+ selection

In [ ]:
# Load data into DataFrame
FPATH = "./data/nfl_teams.csv"
pages = ["Atlanta_Falcons_en.wikipedia.org_mobile-web_all-agents", 
         "Dallas_Cowboys_en.wikipedia.org_mobile-web_all-agents"]

timeseries = get_timeseries(FPATH)
ts_dfs = []
for p in pages:
    print("Preparing timeseries %s" % p)
    df = setup_dataframe(timeseries[p])
    ts_dfs.append(df)

In [ ]:
# Split history (train) and future (test)
sdate = pd.datetime(2017, 7, 10)
ts_data = []
for df in ts_dfs: 
    history, future, y_scale = split_train_test(df, sdate)
    ts_data.append({
        "history": history, "future": future, "y_scale": y_scale
    })
    
print("Extracting features")
ts = ts_data[0] # same feature matrix for all test series  
train_data = extract_features(ts["history"])
test_data = extract_features(ts["future"], changepoints_t=train_data["t_change"])
assert(all(train_data["X"].columns ==  test_data["X"].columns))
assert(all(train_data["t_change"] == test_data["t_change"]))

## Model: FB prophet regression model
We build a regression model similar to [Facebook Prophet](https://peerj.com/preprints/3190/); it combines trend, seasonality and holiday components with non-linear smoothers applied to regressors $t \in \mathbb{Z}^{T}$. 

$$y(t) = g(t) + s(t) + h(t) + \epsilon_{t}  $$

- **Trend** <br/> $$g(t) = (k + \mathbf{a}(t)^{T} \boldsymbol{\delta})t + (m + \mathbf{a}(t)^{T} \boldsymbol{\gamma})$$
    - $k$ is the growth rate (slope)
    - $m$ is the offset (intercept)
    - $S$ changepoints are explicitly defined to allow trend changes at times $s_{j \in {1,2,...,S}}$
        - $\mathbf{a}(t) \in \{0,1\}^{S}$ are changepoint indicators
        - $\delta_{j} \sim Laplace(0,\tau)$ is the change of rate at time $s_{j}$
        - $\gamma_{j}$ is set to $-s_{j}\delta_{j}$ to make the function continuous</br>


- **Seasonality** <br/>
We construct Fourier series to approximate periodic seaonality.
$$s(t) = \sum_{n=1}^{N}(a_{n}cos(\frac{2\pi nt}{P}) + b_{n}sin(\frac{2\pi nt}{P} )) = X(t)  \boldsymbol{\beta}$$
    - $\boldsymbol{\beta} = [a_{1}, b_{1} , ... , a_{N}, b_{N}]^{T}$ and $\boldsymbol{\beta} \sim Normal(0,\sigma^{2})$
    - yearly -> (P = 365.25, N = 10)
    - weekly -> (P = 7, N = 3)
    

- **Holiday/Events** <br/>
Assuming holidays are independnet, we assign each holiday with a parameter $\kappa_{i}$
$$h(t) = Z(t) \boldsymbol{\kappa}$$
    - $Z(t) = [\boldsymbol{1}(t\in D_{1}) , ... , \boldsymbol{1}(t\in D_{L})]$
    - $\boldsymbol{\kappa} \sim Normal(0, \nu^2)$

Before modeling, we transform the raw data and extract features into proper format. The input data includes

- X
    - **t: ** time index
    - **X: ** seasonality vector after fourier transformation
    - **A: ** changepoint vector given time and number of change points
    - **sigmas: ** fixed scale on seasonality priors
- y
    - **y_scaled: ** `maxdiff(log(views))`

In [ ]:
N_TS = len(ts_data)
S = len(train_data["t_change"])
K = train_data["X"].shape[1]

t = tf.placeholder(tf.float32, shape=None, name="t")              # time index
A = tf.placeholder(tf.float32, shape=(None, S), name="A")         # changepoint indicators
t_change = tf.placeholder(tf.float32, shape=(S), name="t_change") # changepoints_t
X = tf.placeholder(tf.float32, shape=(None, K), name="X")         # season vectors
sigmas = tf.placeholder(tf.float32, shape=(K,), name="sigmas")    # scale on seasonality prior
tau = tf.placeholder(tf.float32, shape=(), name="tau")

k = Normal(loc=tf.zeros(1), scale=5.0*tf.ones(1))     # initial slope
m = Normal(loc=tf.zeros(1), scale=5.0*tf.ones(1))     # initial intercept

sigma_obs = Normal(loc=tf.zeros(1), scale=0.5*tf.ones(1))   # noise
delta = Laplace(loc=tf.zeros(S), scale=tau*tf.ones(S))    # changepoint rate adjustment
gamma = tf.multiply(-t_change, delta)
beta = Normal(loc=tf.zeros(K), scale=sigmas*tf.ones(K))      # seasonal
trend_loc = (k + ed.dot(A, delta)) * t + \
            (m + ed.dot(A, gamma))
seas_loc = ed.dot(X, beta)
y = Normal(loc = trend_loc + seas_loc, scale = sigma_obs)

### 1.Inference: HMC
Given train data, the goal is to infer $k,m,\boldsymbol{\delta},\tau,\boldsymbol{\beta}$ and $\sigma$, where k and m are trend model parameters, $\boldsymbol{\delta}$ are latent variables for rate adjustment, $\boldsymbol{\beta}$ are smoothers for seasonality, $\tau$ and $\sigma$ are variance component parameters.

In this analysis, we use Monte Carlo with `ed.HMC` to infer all the latent variables. All training data are passed in for inference and we tune step_size $= 0.0005$, n_steps $= 2$.

In [ ]:
ITR = 5000
kinit, minit = init_km(ts_data[0]["history"])
print("[+] Initial slope / intercept: %f, %f" % (kinit, minit))
qk = Empirical(params=tf.Variable(kinit * tf.ones([ITR, 1])))
qm = Empirical(params=tf.Variable(minit * tf.ones([ITR, 1])))
qsigma_obs = Empirical(params=tf.Variable(tf.ones([ITR, 1])))
qbeta = Empirical(params=tf.Variable(tf.zeros([ITR, K])))
qdelta = Empirical(params=tf.Variable(tf.zeros([ITR, S])))

In [ ]:
posts_dict = {
    k: qk, m: qm, sigma_obs: qsigma_obs,
    beta: qbeta, delta: qdelta}

data_dict = {
    y: ts_data[0]["history"]["y_scaled"].as_matrix(),
    t: train_data["t"],
    X: train_data["X"],
    sigmas: train_data["sigmas"],
    A: train_data["A"],
    t_change: train_data["t_change"],
    tau: 0.05,
}

inference = ed.HMC(posts_dict, data=data_dict)
STEP_SIZE = 5e-4
N_STEPS = 2
inference.run(step_size=STEP_SIZE, n_steps=N_STEPS)

### 1.Prediction

In [ ]:
from prediction import *
nburn = int(ITR / 2)
stride = 10
sess = ed.get_session()
post_params = {
    "k": qk.params.eval()[nburn:ITR:stride],
    "m": qm.params.eval()[nburn:ITR:stride],
    "beta": qbeta.params.eval()[nburn:ITR:stride],
    "delta": qdelta.params.eval()[nburn:ITR:stride]
}
ts = ts_data[0]
pred_df =  make_future_dataframe(ts["history"], ts["future"].shape[0])
pred_df = predict_fixed(pred_df, post_params, test_data)
y_true = ts["future"]["y_scaled"].as_matrix()
_  = evaluate(y_true, pred_df["y"])

plt.plot(ts["future"]["ds"], ts["future"]["y_scaled"], lw=4, color='b')
plt.plot(ts["future"]["ds"], pred_df["y"], color='r')
plt.xticks(rotation=90)
plt.show()
            
# test_data_dict =  {
#     t: test_data["t"],
#     X: test_data["X"],
#     sigmas: test_data["sigmas"],
#     A: test_data["A"],
#     t_change: test_data["t_change"],
#     tau: 0.05,
# }
# CI = [20, 80]
# y_true = ts_data[0]["future"]["y_scaled"]
# y_pred = predict(y, posts_dict, test_data_dict, SAMPLE=500)
# y_pred_mean = np.mean(y_pred, axis=0)[0]
# y_pred_lower = np.percentile(y_pred, CI, axis=0)[0][0]
# y_pred_upper = np.percentile(y_pred, CI, axis=0)[1][0]
# plt.plot(ts_data[0]["future"]["t"], y_true,lw=4, color='b')
# plt.plot(ts_data[0]["future"]["t"], y_pred_mean,lw=4, color='r')
# plt.fill_between(ts_data[0]["future"]["t"], y_pred_lower,y_pred_upper,color='b',alpha=.05)
# plt.show()

# plt.plot(ts_data[0]["future"]["t"],  y_pred_mean - y_true, color='b', label="residual")
# plt.plot(ts_data[0]["future"]["t"],  np.zeros(y_true.shape), label="residual")
# plt.show()

In [ ]:
plt.subplot(411) 
plt.plot(ts["future"]["ds"], pred_df["yearly"])
plt.subplot(412)
plt.plot(ts["future"]["ds"], pred_df["weekly"])
plt.subplot(413)
plt.plot(ts["future"]["ds"], pred_df["trend"])
plt.subplot(414)
plt.plot(ts["future"]["ds"], y_true - pred_df["y"])
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
pred_df.index = pred_df["ds"]
decomposition = seasonal_decompose(y_true - pred_df["y"])
plt.subplot(411) 
plt.plot(decomposition.trend)
plt.subplot(412)
plt.plot(decomposition.seasonal)
plt.subplot(413)
plt.plot(decomposition.resid)
plt.subplot(414)
plt.plot(ts["future"]["ds"], y_true - pred_df["y"])
plt.show()

In [ ]:
pred_df.head()

In [ ]:


history_pred_df = pd.DataFrame({"ds": ts["history"]["ds"].copy(),
                                "t": ts["history"]["t"].copy()})
history_pred_df.reset_index(inplace=True, drop=True)
history_pred_df = predict_fixed(history_pred_df, post_params, train_data)
pred_df.head()
plt.subplot(411) 
plt.plot(ts["history"]["ds"], history_pred_df["yearly"])
plt.subplot(412)
plt.plot(ts["history"]["ds"], history_pred_df["weekly"])
plt.subplot(413)
plt.plot(ts["history"]["ds"], history_pred_df["trend"])
plt.subplot(414)
plt.plot(ts["history"]["ds"], ts["history"]["y_scaled"].as_matrix() - history_pred_df["y"])
plt.show()

### 1.Criticism
- Visualization of the residuals
- Pointwise Evaluation
    - MAPE
    - SMAPE
    - MSE
- PPC

In [ ]:
evaluate(y_true, pred_df["y"])
sns.distplot(pred_df["y"] - y_true)
plt.show()
sns.distplot((pred_df["y"] - y_true)/y_true)
plt.show()

In [ ]:
# Posterior check
sess = ed.get_session()
kmean, kstddev = sess.run([qk.mean(), qk.stddev()])
print("Inferred posterior k: mean = %f, stddev = %f" % (kmean, kstddev))
mmean, mstddev = sess.run([qm.mean(), qm.stddev()])
print("Inferred posterior m: mean = %f, stddev = %f" % (mmean, mstddev))

nburn = int(ITR / 2)
stride = 10
sns.distplot(qk.params.eval()[nburn:ITR:stride])
plt.show()
sns.distplot(qm.params.eval()[nburn:ITR:stride])
plt.show()


In [ ]:
y_post = ed.copy(y, posts_dict)
ty_rep, ty = ed.ppc(lambda xs, zs: tf.reduce_max(tf.cast(xs[y_post], tf.float32)), 
       data={y_post:ts_data[0]["history"]["y_scaled"].as_matrix(), 
            t: train_data['t'],
            A: train_data['A'], 
            X: train_data['X'].as_matrix(), 
            sigmas: train_data['sigmas'], 
            t_change: train_data["t_change"]},n_samples=500)    

# y: ts_data[0]["history"]["y_scaled"].as_matrix(),
#     t: train_data["t"],
#     X: train_data["X"],
#     sigmas: train_data["sigmas"],
#     A: train_data["A"],
#     t_change: train_data["t_change"]

ed.ppc_stat_hist_plot(
    ty[0], ty_rep, stat_name=r'$T \equiv$max', bins=10)
plt.show()

## Round 1.5: 
___

### Improvements
- motify init k and m
- remove holiday features why us public holiday not works, require in-domain knowledge
- tau from fixed value to latent variable

## Round 2: 
___
### 2.Model
We modify the model by creating global latent variable inferring local seasonality  
- Local latend variable
    - 
- Global latent variable
    - `gbeta`
    
Given train data, the goal is to infer $k,m,\boldsymbol{\delta},\tau,\boldsymbol{\beta}$ and $\sigma$, where k and m are trend model parameters, $\boldsymbol{\delta}$ are latent variables for rate adjustment, $\boldsymbol{\beta}$ are smoothers for seasonality, $\tau$ and $\sigma$ are variance component parameters.

In [ ]:
def visualize_results(ts_data, predictions, metrics):
    for i, df in enumerate(ts_dfs):
        plt.plot(ts_data[i]["future"]["ds"], ts_data[i]["future"]["y_scaled"])
        plt.plot(predictions[i]["ds"], predictions[i]["y_scaled_pred"], '#2ca02c')
        plt.show()  
    m_pd = pd.DataFrame.from_dict(metrics)
    m_pd.loc['mean'] = m_pd.mean()
    print(m_pd)
    print()

def visualize_cross_validation(ts_dfs, predictions, metrics):
    for i, df in enumerate(ts_dfs):
        df = df[df["ds"] > pd.datetime(2016,6,1)]
        plt.plot(df["ds"], df["y"])
        for pred in predictions:
            plt.plot(pred[i]["ds"], pred[i]["y_pred"], '#2ca02c')
        plt.show()
    
    metrics_df = pd.DataFrame(columns=['start', 'end', 'MAPE_avg', 'SMAPE_avg'])
    for i, m_cutoff in enumerate(metrics):
        dmin, dmax = predictions[i][0]["ds"].min(), predictions[i][0]["ds"].max()
        avg_mape_scaled = np.mean([m["MAPE"] for m in m_cutoff])
        avg_smape_scaled = np.mean([m["SMAPE"] for m in m_cutoff])
        metrics_df = metrics_df.append({"start": dmin,
                                        "end": dmax,
                                        "MAPE_avg": avg_mape_scaled, 
                                        "SMAPE_avg": avg_smape_scaled}, ignore_index=True)
    
    print(metrics_df)

In [ ]:
#%%capture
results = []
models_test = [Model1(), Model3()]
print(train_data.keys())
for model in models_test:
    p, m = pipeline(ts_data, model, train_data, test_data, 
                    ITR=5000, N_STEPS=2, STEP_SIZE=5e-4)
    results.append({"predictions": p, "metrics": m})

In [ ]:
#results[0]["predictions"][0]

In [ ]:
import itertools
colors = itertools.cycle(('r', 'y', 'b', 'g'))

for i, ts in enumerate(ts_data):
    plt.plot(ts["future"]["t"], ts["future"]["y_scaled"], lw=3, color='g', label="True")
    for j, r in enumerate(results):
        df = r["predictions"][i] 
        c = next(colors)
        plt.plot(ts["future"]["t"], df["y"], label="Model %d" % j, color=c)
        #lt.fill_between(ts_data[i]["future"]["t"], df["y_scaled_pred_lower"], df["y_scaled_pred_upper"], color=c, alpha=.05)
    plt.legend(loc=2, prop={'size': 24})
    plt.show()
print()

In [ ]:
def plot_composition(df, y_true): 
    df.index = df["ds"]
    decomposition = seasonal_decompose(y_true - df["y"])
    plt.subplot(411) 
    plt.plot(decomposition.trend)
    plt.subplot(412)
    plt.plot(decomposition.seasonal)
    plt.subplot(413)
    plt.plot(decomposition.resid)
    plt.subplot(414)
    plt.plot(df["ds"], y_true - pred_df["y"])
    plt.show()
        
    
for i, ts in enumerate(ts_data):
    for j, r in enumerate(results):
        df = r["predictions"][i]
        plot_composition(df, ts["future"]["y_scaled"].as_matrix())

## Conclusion and Lessions Learned